In [5]:
from braket.aws import AwsDevice
from braket.aws import AwsQuantumJob, AwsSession
from braket.jobs.config import InstanceConfig

import boto3

import os
import json
import time
import altair as alt
import pandas as pd
import numpy as np

In [126]:
import copy

c = copy.deepcopy

def get_key(single_dict):
    for k in single_dict.keys():
        return k

def parse_params(params_list, hp, hp_list):
    params = params_list[0]
    k = get_key(params)
    ps = params[k]
    for p in ps:
        hp[k] = p
        if len(params_list) == 1:
            hp_list.append(c(hp))
        else:
            parse_params(params_list[1:], hp, hp_list)

def get_quantum_device(device_name):
    device_arn = "arn:aws:braket:::device/quantum-simulator/amazon/sv1"
    try:
        device = AwsDevice.get_devices(names=[device_name])
        device_arn = device[0].arn
    except Exception as e:
        print(f"fail to get {device_name}: {e}, use sv1 instead")
    return device_arn

def upload_data(dir_name, suffix_check, aws_session=AwsSession()):
    stream_s3_uri = aws_session.construct_s3_uri(aws_session.default_bucket(), dir_name)
    return_path = None
    
    def _check_upload(file_name, check_list):
        file_end = file_name.split('.')[-1]
        if file_end in check_list:
            path = f"{stream_s3_uri}/" + file_name.split('/')[-1]
            aws_session.upload_to_s3(file_name, path)
            
    if os.path.isdir(dir_name):
        dir_list = os.listdir(dir_name)
        for file_name in dir_list:
            _check_upload(os.path.join(dir_name,file_name), suffix_check)
        return_path = stream_s3_uri
    else:
        _check_upload(file_name, suffix_check)
        single_file_name = file_name.split('/')[-1]
        return_path = f"{stream_s3_uri}/{single_file_name}"
        
    return return_path

def queue_check(jobs):
    queue_count = 0
    running_count = 0
    check_pass = True
    for job in jobs:
        # print(f"job state {job.state()}")
        if job.state() == "QUEUED":
            queue_count = queue_count + 1
        if job.state() == "RUNNING":
            running_count = running_count + 1
        if queue_count == 4 or running_count == 4:
            check_pass = False
    
    print(f"queue_count {queue_count}, running_count {running_count}")
    
    return check_pass

def get_result(result, target, dm):
    return [dm, result["time"], target]

def get_dm(target):
    file_name = './test-folding/' + target + '.fasta.txt'
    str_len = -100
    with open(file_name) as file:
        fasta_lines = file.readlines()
        str_len = len(fasta_lines[1])
    return str_len

def display_results(results, experiments_params):
    sorted_results = {}
    
    for device in experiments_params["params"][4]["device"]:
        sorted_results[str(device)] = []
        # for target in results[0].keys():
        #     sorted_results[str(device)][target] = []
    
    max_offset = 10
    for result in results:
        for target in result.keys():
            device = result[target]["hypermeter"]["device"]
            
            dm = get_dm(target)

            if len(sorted_results[device]) == 0:
                sorted_results[device].append(get_result(result[target],target,dm))
                continue

            last_idx = 0
            for idx, sorted_result in enumerate(sorted_results[device]):
                sorted_dm = float(sorted_result[0])

                last_sorted_dm = float(sorted_results[device][last_idx][0])

                if last_sorted_dm < dm and dm < sorted_dm:
                    sorted_results[device].insert(idx,get_result(result[target],target,dm))
                    break
                elif dm < last_sorted_dm and idx == 0:
                    sorted_results[device].insert(last_idx,get_result(result[target],target,dm))
                    break
                elif dm > sorted_dm and idx == len(sorted_results[device])-1:
                    sorted_results[device].insert(idx+1,get_result(result[target],target,dm))

                last_idx = idx
    # print(f"sorted result {sorted_results}")
    return sorted_results

In [117]:
# parameters for experiments
experiment_name = "rna-folding"
data_path = "rna-folding-data"
suffix_check = ["txt"]
experiments_params =  {
    "version": "1",
    "params": [
        # {"PKP": [-1.0, -0.5, 0.0, 0.5, 1.0]},
        {"PKP": [-1.0]},
        {"S": [1]},
        {"O": [1000000]},
        {"shots": [10000]},
        {"device": [{"qc": "null", "cc": "ml.m5.large"},{"qc": "null", "cc": "ml.m5.4xlarge"}]}
    ]
}

hp = {}
hybrid_job_params = []
parse_params(experiments_params['params'], hp, hybrid_job_params)

print(f"parameters for experiments: \n {hybrid_job_params}")

parameters for experiments: 
 [{'PKP': -1.0, 'S': 1, 'O': 1000000, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.large'}}, {'PKP': -1.0, 'S': 1, 'O': 1000000, 'shots': 10000, 'device': {'qc': 'null', 'cc': 'ml.m5.4xlarge'}}]


In [119]:
# Upload dataset to S3
s3_path = upload_data(data_path, suffix_check)
print(f"upload data to s3 path: {s3_path}")

upload data to s3 path: s3://amazon-braket-us-east-1-002224604296/rna-folding-data


In [120]:
account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.client('s3').meta.region_name
image_name = f"amazon-braket-{experiment_name}-jobs"
image_uri = f"{account_id}.dkr.ecr.{region}.amazonaws.com/{image_name}:latest"

print(f"the hybrid job image for {account_id} in region {region}: {image_uri}")

# # Uncomment the following code if you want to rebuild the container
# !sh build_and_push.sh {image_name}

the hybrid job image for 002224604296 in region us-east-1: 002224604296.dkr.ecr.us-east-1.amazonaws.com/amazon-braket-rna-folding-jobs:latest


In [134]:
hybrid_jobs_json = f"{experiment_name}-hybrid-jobs.json"
print(f"job info will be saved in {hybrid_jobs_json}")

job info will be saved in rna-folding-hybrid-jobs.json


In [122]:
# Long runnning cell due to Burst rate of CreateJob requests < 5 RPS
# sudo apt-get install python-prctl at first
# https://stackoverflow.com/questions/34361035/python-thread-name-doesnt-show-up-on-ps-or-htop
from threading import Thread
import threading
import setproctitle

def launch_hybrid_jobs(hybrid_job_params=hybrid_job_params, hybrid_jobs_json=hybrid_jobs_json):
    setproctitle.setproctitle(threading.currentThread().name)
    # parse evaluation parameters and trigger hybrid jobs:
    jobs = []
    names = []

    job_name = f"{experiment_name}-job"

    for job_param in hybrid_job_params:
        PKP = job_param['PKP']
        S = job_param['S']
        O = job_param['O']
        quantum_device = get_quantum_device(job_param['device']['qc'])
        classical_device = job_param['device']['cc']

        device_name = classical_device.replace(".","-")
        device_name = device_name.replace("x","")
        
        name = f"{experiment_name}-PKP-{str(PKP).replace('.','')}-{device_name}-" + str(int(time.time()))
        print(f"name is {name}")

        tmp_job = AwsQuantumJob.create(
            device=quantum_device,
            source_module=f"{experiment_name}",
            entry_point=f"{experiment_name}.{job_name}:main",
            job_name=name,
            hyperparameters=job_param,
            input_data=s3_path,
            instance_config=InstanceConfig(instanceType=classical_device),
            image_uri=image_uri,
            wait_until_complete=False,
        )
        
#         from braket.jobs.local import LocalQuantumJob
        
#         tmp_job = LocalQuantumJob.create(
#             device=quantum_device,
#             source_module=f"{experiment_name}",
#             entry_point=f"{experiment_name}.{job_name}:main",
#             hyperparameters=job_param,
#             input_data=s3_path,
#             image_uri=image_uri,
#         )   
        
        
        print(f"Finish create {experiment_name} with PKP {PKP}, S {S} , O {O} and device {device_name}")

        jobs.append(tmp_job)
        names.append(name)


        while not queue_check(jobs):
            time.sleep(5)
    jobs_arn = []

    for job in jobs:
        jobs_arn.append(job.arn)

    jobs_states = {
        "experiment_name": experiment_name,
        "hybrid-jobs-arn": jobs_arn,
        "names": names
    }
    
    
    # save hybrid job arn for further analysis
    json_object = json.dumps(jobs_states, indent=4)

    with open(hybrid_jobs_json, "w") as outfile:
        outfile.write(json_object)
        
    print(f"Finish launch all the hybrid jobs and save all the files")

# remove existing hybrid_jobs_json file
!rm {hybrid_jobs_json}

t = Thread(target=launch_hybrid_jobs, name="launch-hybrid-job", daemon=True).start()

# launch_hybrid_jobs()

rm: cannot remove 'rna-folding-hybrid-jobs.json': No such file or directory
fail to get null: list index out of range, use sv1 instead
name is rna-folding-PKP--10-ml-m5-large-1673244331
Finish create rna-folding with PKP -1.0, S 1 , O 1000000 and device ml-m5-large
queue_count 1, running_count 0
fail to get null: list index out of range, use sv1 instead
name is rna-folding-PKP--10-ml-m5-4large-1673244336
Finish create rna-folding with PKP -1.0, S 1 , O 1000000 and device ml-m5-4large
queue_count 1, running_count 1
Finish launch all the hybrid jobs and save all the files


In [123]:
# run the following scripts to check the created threads
!ps -aux | grep launch-hybrid-job

ubuntu     85131  0.0  1.3 1667824 438060 ?      Ssl  Jan05   0:25 launch-hybrid-job
ubuntu     90748  0.0  1.5 1867392 494316 ?      Ssl  Jan06   1:30 launch-hybrid-job
ubuntu    132026  0.0  0.0   8748  3300 pts/0    Ss+  06:14   0:00 /bin/bash -c ps -aux | grep launch-hybrid-job
ubuntu    132028  0.0  0.0   8176   724 pts/0    S+   06:14   0:00 grep launch-hybrid-job


In [135]:
# run the following code to test whether all the jobs finish
results = []
if os.path.exists(hybrid_jobs_json):
    # recover hybrid jobs and show result
    jobs_states_load = None
    with open(hybrid_jobs_json, "r") as outfile:
        jobs_states_load = json.load(outfile)

    completed_jobs_arn = set()

    for job_name, job_arn in zip(jobs_states_load["names"], jobs_states_load["hybrid-jobs-arn"]):
        current_job = AwsQuantumJob(job_arn)
        print(f"the state of job {job_name} is : {current_job.state()}")
        if current_job.state() == 'COMPLETED':
            completed_jobs_arn.update({job_arn})

    whole_jobs_num = len(jobs_states_load["names"])

    if len(completed_jobs_arn) == whole_jobs_num:
        print(f"all jobs completed")
        for job_arn in completed_jobs_arn:
            current_job = AwsQuantumJob(job_arn)
            results.append(current_job.result())
        # display results
        results = display_results(results, experiments_params)
else:
    print(f"JSON file for job arns not generated! please wait for the thread(launch-hybrid-job) to finish")

the state of job rna-folding-PKP--10-ml-m5-large-1673244331 is : RUNNING
the state of job rna-folding-PKP--10-ml-m5-4large-1673244336 is : RUNNING


In [136]:
rename_result = {}
device_list = []
x_list = []
y_list = []
for k,vs in results.items():
    k = k.replace("\'","\"")
    dict_k = json.loads(k)
    device_name = None
    if dict_k['qc'] == 'null':
        device_name = dict_k['cc']
    else:
        device_name = dict_k['qc']
    for v in vs:
        device_list.append(device_name)
        x_list.append(v[0])
        y_list.append(v[1])
source = pd.DataFrame({
    "Sequence Length": np.array(x_list),
    "Time to Solution": np.array(y_list),
    "Device": np.array(device_list),
})

alt.Chart(source).mark_line(point = True).encode(
    x='Sequence Length',
    y='Time to Solution',
    color='Device',
).properties(
    title = f"{experiment_name} experiments",
    width = 700,
    height = 600,
).interactive()

AttributeError: 'list' object has no attribute 'items'